In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.cross_validation import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from math import sqrt

from scipy import randn

In [ ]:
data = pd.read_csv('data/prices.txt')

In [ ]:
data.head()

In [ ]:
plot1 = data.plot(kind='scatter', x='area', y='price', c='rooms', grid=True)
#plot1.set_facecolor("gray")

In [ ]:
data.plot(kind = 'scatter', x = 'rooms', y='price')

In [ ]:
data.plot(kind = 'scatter', x = 'area', y='price')

In [ ]:
X = data[['area', 'rooms']]
y = np.array(list(map(float, data['price'])))

In [ ]:
def standard_transform(x):
    return StandardScaler().fit_transform(x)

def append_one(x):
    return np.append(np.ones(len(x)).reshape((len(x), 1)), x, axis=1)

In [ ]:
scalerX = StandardScaler()
scalerY = StandardScaler()
initX = X
X = scalerX.fit_transform(X)
X = append_one(X)
initX = append_one(initX)

y = scalerY.fit_transform(y.reshape(len(y), 1)).reshape(len(y))


In [ ]:
def first_price(p):    
    return scalerY.inverse_transform([p])[0]    
    
def get_mse(regressor, xs, ys):    
    predicted = np.apply_along_axis(regressor.predict, 1, xs)
    return sqrt((
        (scalerY.inverse_transform(predicted) - scalerY.inverse_transform(ys)) ** 2)
                .mean())


In [ ]:
class LinearRegression:
    def __init__(self, w = None, linearCoef = 0.5, numIter = 100, eps = 1e-9):
        self.baseW = w
        self.linearCoef = linearCoef   
        self.numIter = numIter
        self.eps = eps
        
    def _grad_descend(self):
        Xt = self.X.transpose()
        m = self.X.shape[0]
        mse = None
        w = self.baseW
        for i in range(self.numIter):
            h = np.dot(self.X, w)
            diff = h - self.y
            grad = 2 * np.dot(Xt, diff) / m            
            w -= (self.linearCoef[i] * grad)
            if (self.linearCoef[i] * np.sum(grad ** 2) < self.eps):                
                break
            
        self.w = w      
    
    def fit(self, X, y):
        self.X = X
        self.y = y      
        self._grad_descend()

    def predict(self, X):               
        return np.dot(X, self.w)
    
    def weights(self):
        return self.w


In [ ]:
%%time

m, n = X.shape
w = np.ones(n)
numIter = 100
linearCoef = [0.5] * numIter
linearReg = LinearRegression(w = w, linearCoef = linearCoef, numIter = numIter)
linearReg.fit(X, y)

print (linearReg.weights())
print (get_mse(linearReg, X, y))

In [ ]:
def show_separator(weights):
    tX = X.transpose()
    for i, x in enumerate(initX.transpose()[1:], 1):                
        plt.scatter(x, scalerY.inverse_transform(y)) 
        plt.plot([x.min(), x.max()], [first_price(weights[0] + weights[i] * tX[i].min()),
                                      first_price(weights[0] + weights[i] * tX[i].max())])      
        plt.ylabel('Price')
        plt.xlabel('%s' % ['area', 'rooms'][i-1])
        plt.show()

In [ ]:
linW = linearReg.weights()
show_separator(linW)

In [ ]:
np.random.seed(53)

class GeneticReg:
    def __init__(self, numIter = 100, popSize = 50, mutationCount = 5):        
        self.numIter = numIter
        self.popSize = popSize
        self.mutationCount = mutationCount        
        
    def mutation(self, pop):            
        return pop + randn(*pop.shape)
    
    def gen_optimize(self):
        m, n = self.X.shape
        pop = np.zeros((self.popSize, n))
        pops = []
        mses = []        
       
        for i in range(self.numIter):
            pops = pop.copy()
            for j in range(self.mutationCount):
                pops = np.append(pops, self.mutation(pop), axis=0)
                        
            mses = []
            for j, single_pop in enumerate(pops):
                h = np.dot(X, single_pop)
                diff = h - y
                mse = np.sum(diff ** 2) / (m)
                mses.append((mse, j))
            mses = sorted(mses)
            pop = pops[mses[0][1]].reshape(1, len(pops[mses[0][1]]))
            
            for _, j in mses[1:self.popSize]:
                pop = np.append(pop, pops[j].reshape(1, len(pops[j])), axis=0)                                    
        
        self.w = pops[mses[0][1]]
    
    def fit(self, X, y):
        self.X = X
        self.y = y      
        self.gen_optimize()

    def predict(self, X):               
        return np.dot(X, self.w)
    
    def weights(self):
        return self.w

In [ ]:
#iterAr = [100, 500, 1000]
#for ni in iterAr:
    #mseGen = []
    #for i in range(10):        
        #genTest = GeneticReg(numIter = ni, popSize = 30, mutationCount = 5)
        #genTest.fit(X, y)
        #mseGen.append(get_mse(genTest, X, y))
    #print ("Iter count: %d" % ni)
    #print ("Average: %d" % np.mean(mseGen))
    #print ("Minimum: %d" % np.min(mseGen))
    #print ("Maximum: %d" % np.max(mseGen))
    

In [ ]:
%%time

genReg = GeneticReg(numIter = 500, popSize = 50, mutationCount = 5)
genReg.fit(X, y)
print (get_mse(genReg, X, y))
print (genReg.weights())

In [ ]:
genW = genReg.weights()
show_separator(genW)

In [ ]:
exX = [[2100., 3.]]
exX = append_one(scalerX.transform(exX))

print (first_price(linearReg.predict(exX)))
print (first_price(genReg.predict(exX)))